In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
hdb_df = pd.read_csv('data/hdb_combined_clean.csv', low_memory=False)

In [3]:
hdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917393 entries, 0 to 917392
Data columns (total 22 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   month_of_sale                 917393 non-null  object 
 1   town                          917393 non-null  object 
 2   flat_type                     917393 non-null  object 
 3   block                         917393 non-null  object 
 4   street_name                   917393 non-null  object 
 5   storey_range                  917393 non-null  object 
 6   floor_area_sqm                917393 non-null  float64
 7   flat_model                    917393 non-null  object 
 8   lease_commence_date           917393 non-null  int64  
 9   resale_price                  917393 non-null  float64
 10  address                       917393 non-null  object 
 11  latitude                      917393 non-null  float64
 12  longitude                     917393 non-nul

#  Regression 

In [ ]:
# # marking the dataset number
# conditions = [
#     (hdb_df['year_of_sale'] >= 1990) & (hdb_df['year_of_sale'] <= 1999),
#     (hdb_df['year_of_sale'] >= 2000) & (hdb_df['year_of_sale'] <= 2012),
#     (hdb_df['year_of_sale'] >= 2013) & (hdb_df['year_of_sale'] <= 2014),
#     (hdb_df['year_of_sale'] >= 2015) & (hdb_df['year_of_sale'] <= 2016),
#     (hdb_df['year_of_sale'] >= 2017)
# ]
# labels = [1, 2, 3, 4, 5]

# hdb_df['source_dataset'] = np.select(conditions, labels, default=np.nan)

In [ ]:
# hdb_df['source_dataset'].value_counts()

In [ ]:
# # Create a 2x3 grid for displaying plots (2 plots per row)
# fig, axes = plt.subplots(3, 2, figsize=(14, 12))  # 3 rows, 2 columns
# axes = axes.flatten()  # Flatten the 2D array of axes for easier indexing

# # Loop through the 5 source datasets
# for i in range(1, 6):
#     hdb_df_subset = hdb_df[hdb_df['source_dataset'] == i]  # Filter for specific source dataset
    
#     # Compute the correlation between floor_area_sqm and price/m2
#     correlation = hdb_df_subset.corr().loc['floor_area_sqm', 'resale_price']
    
#     # Plot the scatter plot on the corresponding subplot
#     axes[i-1].scatter(hdb_df_subset['floor_area_sqm'], hdb_df_subset['resale_price'])
#     axes[i-1].set_title(f"Source Dataset {i} (Corr: {correlation:.2f})")
#     axes[i-1].set_xlabel("Floor Area (sqm)")
#     axes[i-1].set_ylabel("Resale Price")

# # Remove the last empty subplot (since we have 5 datasets but 6 subplots)
# fig.delaxes(axes[-1])

# # Adjust the layout so the plots don't overlap
# plt.tight_layout()
# plt.show()


3 and 4 got lesser outliers

In [7]:
# hdb_df2 = hdb_df[hdb_df['year_of_sale']==2020]
# hdb_df2.shape

## Bool encoding

In [8]:
# Convert categorical columns to numerical
categorical_cols = ['town', 'flat_type', 'flat_model']  # Replace with relevant column names
df_encoded = pd.get_dummies(hdb_df, columns=categorical_cols, drop_first=True)

In [9]:
df_encoded.drop(['month_of_sale', 'block', 'street_name', 'storey_range', 'lease_commence_date', 'address',
             'latitude', 'longitude','price_per_sqm'],
            axis=1, inplace=True)

In [10]:
df_encoded = df_encoded.drop_duplicates()

In [11]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 906790 entries, 0 to 917392
Data columns (total 62 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   floor_area_sqm                     906790 non-null  float64
 1   resale_price                       906790 non-null  float64
 2   nearest_supermarket_distance       906790 non-null  float64
 3   nearest_school_distance            906790 non-null  float64
 4   nearest_mrt_distance               906790 non-null  float64
 5   nearest_hawkers_distance           906790 non-null  float64
 6   cbd_distance                       906790 non-null  float64
 7   year_of_sale                       906790 non-null  int64  
 8   calculated_remaining_lease         906790 non-null  float64
 9   storey_median                      906790 non-null  int64  
 10  town_BEDOK                         906790 non-null  uint8  
 11  town_BISHAN                        9067

## Weights

In [54]:
# Create a dictionary to assign specific weights to each town
town_encoding = {
    'TAMPINES': 1, 'YISHUN': 2, 'BEDOK': 3, 'JURONG WEST': 4, 
    'WOODLANDS': 5, 'ANG MO KIO': 6, 'HOUGANG': 7, 'BUKIT BATOK': 8, 
    'CHOA CHU KANG': 9, 'BUKIT MERAH': 10, 'PASIR RIS': 11, 
    'SENGKANG': 12, 'TOA PAYOH': 13, 'QUEENSTOWN': 14, 
    'GEYLANG': 15, 'CLEMENTI': 16, 'BUKIT PANJANG': 17, 'KALLANG/WHAMPOA': 18,
    'JURONG EAST': 19, 'SERANGOON': 20, 'BISHAN': 21, 
    'PUNGGOL': 22, 'SEMBAWANG': 23, 'MARINE PARADE': 24, 
    'CENTRAL AREA': 25, 'BUKIT TIMAH': 26, 'LIM CHU KANG': 27
}

# Create a dictionary to assign specific weights to each flat type
flat_type_encoding = {
    '1 ROOM': 1,
    '2 ROOM': 2,
    '3 ROOM': 3,
    '4 ROOM': 4,
    '5 ROOM': 5,
    'EXECUTIVE': 6,
    'MULTI GENERATION': 7,
}

# Create a dictionary to assign specific weights to each flat model
flat_model_encoding = {
    'Model A': 1,
    'Improved': 2,
    'New Generation': 3,
    'Simplified': 4,
    'Premium Apartment': 5,
    'Standard': 6,
    'Apartment': 7,
    'Maisonette': 8,
    'Model A2': 9,
    'DBSS': 10,
    'Model A-Maisonette': 11,
    'Adjoined flat': 12,
    'Terrace': 13,
    'Multi Generation': 14,
    'Type S1': 15,
    'Type S2': 16,
    'Improved-Maisonette': 17,
    '2-Room': 18,
    'Premium Apartment Loft': 19,
    'Premium Maisonette': 20,
    '3Gen': 21,
}

# Apply encoding to the DataFrame using the created dictionaries
hdb_df['town'] = hdb_df['town'].map(town_encoding)
hdb_df['flat_type'] = hdb_df['flat_type'].map(flat_type_encoding)
hdb_df['flat_model'] = hdb_df['flat_model'].map(flat_model_encoding)


In [55]:
hdb_df.drop(['month_of_sale', 'block', 'street_name', 'storey_range', 'lease_commence_date', 'address',
             'latitude', 'longitude', 'price_per_sqm'],
            axis=1, inplace=True)

In [56]:
hdb_df = hdb_df.drop_duplicates()

In [57]:
hdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 906790 entries, 0 to 917392
Data columns (total 13 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   town                          906790 non-null  int64  
 1   flat_type                     906790 non-null  int64  
 2   floor_area_sqm                906790 non-null  float64
 3   flat_model                    906790 non-null  int64  
 4   resale_price                  906790 non-null  float64
 5   nearest_supermarket_distance  906790 non-null  float64
 6   nearest_school_distance       906790 non-null  float64
 7   nearest_mrt_distance          906790 non-null  float64
 8   nearest_hawkers_distance      906790 non-null  float64
 9   cbd_distance                  906790 non-null  float64
 10  year_of_sale                  906790 non-null  int64  
 11  calculated_remaining_lease    906790 non-null  float64
 12  storey_median                 906790 non-nul

In [58]:
hdb_df.describe()

,town,flat_type,floor_area_sqm,flat_model,resale_price,nearest_supermarket_distance,nearest_school_distance,nearest_mrt_distance,nearest_hawkers_distance,cbd_distance,year_of_sale,calculated_remaining_lease,storey_median
count,906790.000000,906790.000000,906790.000000,906790.000000,9.067900e+05,906790.000000,906790.000000,906790.000000,906790.000000,906790.000000,906790.000000,906790.000000,906790.000000
mean,9.391763,4.019978,95.884599,2.935936,3.195776e+05,293.734470,338.105819,781.043318,1436.744043,11741.283137,2006.057924,81.174875,7.684465
std,6.665466,0.947675,25.815187,2.168410,1.683205e+05,157.620591,211.820193,419.554655,1344.048773,4383.176785,9.204639,10.642207,4.819761
min,1.000000,1.000000,28.000000,1.000000,5.000000e+03,0.000000,42.554861,0.000000,0.000000,721.958596,1990.000000,41.000000,2.000000
25%,4.000000,3.000000,73.000000,1.000000,1.940000e+05,183.611994,198.916384,471.343892,403.688925,8713.428694,1999.000000,74.000000,5.000000
50%,8.000000,4.000000,93.000000,2.000000,2.960000e+05,276.000730,290.642355,711.202633,950.302431,12451.615101,2005.000000,83.000000,8.000000
75%,15.000000,5.000000,113.000000,4.000000,4.150000e+05,383.008811,424.518365,1013.899247,2068.120141,15119.407302,2013.000000,90.000000,11.000000
max,27.000000,7.000000,307.000000,21.000000,1.568888e+06,4470.303250,5144.585152,5466.706999,7505.538603,22074.073156,2024.000000,99.000000,50.000000


## VIF

In [20]:
# Calculate VIF
hdb_vif = hdb_df.drop(columns=['resale_price'])

from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

result = calc_vif(hdb_vif.select_dtypes(include=['int64', 'float64'])).sort_values(by='VIF', ascending=False)
print(result)

                       variables         VIF
1                      flat_type  243.672166
2                 floor_area_sqm  177.947865
9                   year_of_sale   89.887808
10    calculated_remaining_lease   74.345665
8                   cbd_distance   15.779908
6           nearest_mrt_distance    4.672150
4   nearest_supermarket_distance    4.599037
0                           town    4.289246
5        nearest_school_distance    3.887053
7       nearest_hawkers_distance    3.795117
11                 storey_median    3.734591
3                     flat_model    3.587618


high multicollinearity means should remove the variable to improve the results

## Models

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split features and target (assuming 'resale_price' is the target)
X = hdb_df.drop(columns=['resale_price'])
y = hdb_df['resale_price']

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features (only for models sensitive to scale like linear regression or neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)

print("Linear Regression:")
print("R² score:", r2_score(y_test, y_pred_lr))
print("RMSE:", root_mean_squared_error(y_test, y_pred_lr))

Linear Regression:
R² score: 0.8184024614367942
RMSE: 71452.45656831685


In [ ]:
# df_encoded
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
import time

# Dictionary of models to evaluate with fewer estimators for faster training
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
#     "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),  # Consider using early stopping here
#     "XGBoost": XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1),
#     "Lasso": Lasso(max_iter=10000),
#     "Ridge": Ridge(max_iter=10000),
}

# Loop through models, fit, predict, and print scores with progress bar
start_time = time.time()
for name, model in tqdm(models.items(), desc="Training models", unit="model"):
    # Fit the model
    model.fit(X_train_scaled, y_train)
    
    # Predict on test data
    y_pred = model.predict(X_test_scaled)
    
    # Calculate performance metrics
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Print model performance
    print(f"{name}:")
    print(f"R² score: {r2:.4f}")
    print(f"RMSE: {rmse:.4f}\n")

# Display total runtime
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total time taken: {elapsed_time:.2f} seconds")


In [ ]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("model_rf.pkl","wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [16]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# # Assuming your dataset is loaded into hdb_df
# numerical_features = ['floor_area_sqm', 'storey_median', 'calculated_remaining_lease', 
#                       'nearest_mrt_distance', 'nearest_school_distance', 
#                       'nearest_supermarket_distance', 'nearest_hawkers_distance']

# numerical_features2 = ['floor_area_sqm', 'storey_median', 'nearest_hawkers_distance']
# target = 'resale_price'
# target2 = 'price/m2'

# # Split data into features (X) and target (y)
# X = df_encoded[numerical_features]
# y = df_encoded[target2]

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Scaling numerical features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)


In [17]:
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import root_mean_squared_error, r2_score

# lr_model = LinearRegression()
# lr_model.fit(X_train_scaled, y_train)
# y_pred_lr = lr_model.predict(X_test_scaled)

# print("Linear Regression:")
# print("R² score:", r2_score(y_test, y_pred_lr))
# print("RMSE:", root_mean_squared_error(y_test, y_pred_lr))


In [18]:
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.svm import SVR
# from xgboost import XGBRegressor
# from sklearn.linear_model import Lasso, Ridge
# from sklearn.metrics import mean_squared_error, r2_score
# import numpy as np

# # Dictionary of models to evaluate
# models = {
#     "Random Forest": RandomForestRegressor(n_estimators=500, random_state=42),
#     "Gradient Boosting": GradientBoostingRegressor(n_estimators=500, random_state=42),
#     "SVR": SVR(),
#     "XGBoost": XGBRegressor(n_estimators=500, random_state=42),
#     "Lasso": Lasso(),
#     "Ridge": Ridge()
# }

# # Loop through models, fit, predict, and print scores
# for name, model in models.items():
#     # Fit the model
#     model.fit(X_train_scaled, y_train)
    
#     # Predict on test data
#     y_pred = model.predict(X_test_scaled)
    
#     # Calculate performance metrics
#     r2 = r2_score(y_test, y_pred)
#     rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
#     # Print model performance
#     print(f"{name}:")
#     print(f"R² score: {r2:.4f}")
#     print(f"RMSE: {rmse:.4f}\n")


In [19]:
# import matplotlib.pyplot as plt

# # For Random Forest
# importances = rf_model.feature_importances_
# feature_names = numerical_features

# # Plot feature importance
# plt.figure(figsize=(10, 6))
# plt.barh(feature_names, importances)
# plt.xlabel('Feature Importance')
# plt.ylabel('Feature')
# plt.title('Feature Importance for Random Forest Regressor')
# plt.show()
